In [ ]:
from typing import List, Callable, Tuple
from random import choices
from collections import namedtuple

In [ ]:
Genome = List[int]
Population = List[Genome]
Thing = namedtuple('Thing', ['name', 'value', 'weight'])
FitnessFunc = Callable[[Genome], int]
PopulateFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]

In [ ]:
things = [
    Thing('Laptop', 500, 2200),
  Thing('Headphones', 150, 160),
  Thing('Coffee Mug', 60, 350),
    Thing('Notepad', 40, 333),
  Thing('Water Bottle', 30, 192),
]

more_things = [
  Thing('Mints', 5, 25),
  Thing('Socks', 10, 38),
  Thing('Tissues', 15, 80),
  Thing('Phone', 500, 200),
  Thing('Baseball Cap', 100, 70),
] + things

In [ ]:
def generate_genome(length: int) -> Genome:
  return choices([0,1], k=length)

In [ ]:
def generate_population(size: int, genome_length: int) -> Population:
  return [generate_genome(genome_length) for _ in range(size)]

In [ ]:
def fitness(genome: Genome, things: [Thing],weight_limit:int) -> int:
  if len(genome) != len(things):
    raise ValueError("genome and things must be of the same length")

  weight = 0
  value = 0

  for i, thing in enumerate(things):
    if genome[i] == 1:
      weight += thing.weight
      value += thing.value

      if weight > weight_limit:
        return 0

  return value

In [ ]:
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
  return choices(
    population=population,
    weights=[fitness_func(genome) for genome in population],
    k=2
  )

In [ ]:
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
  if len(a) != len(b):
    raise ValueError("Genomes a and b must be of the same length")

  length = len(a)
  if length < 2:
    return a, b

  p = randint(1, length - 1)
  return a[0:p] + b[p:], b[0:p] + a[p:]

In [ ]:
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
  for _ in range(num):
    index = randrange(len(genome))
    genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
  return genome

In [ ]:
from functools import partial
import time


In [ ]:
def run_evolution(
    population_func: PopulateFunc,
    fitness_func: FitnessFunc,
    fitness_limit: int,
    selection_func: SelectionFunc = selection_pair,
    crossover_func: CrossoverFunc = single_point_crossover,
    mutation_func: MutationFunc = mutation,
    generation_limit: int = 100
) -> Tuple[Population, int]:
  population = population_func()
  for i in range(generation_limit):
    population = sorted(
      population,
      key=lambda genome: fitness_func(genome),
      reverse=True
    )
    if fitness_func(population[0]) >= fitness_limit:
      break
    next_generation = population[0:2]

    for j in range(int(len(population) /2 - 1)):
      parents = selection_func(population, fitness_func)
      offspring_a, offspring_b = crossover_func(parents[0], parents[1])
      offspring_a = mutation_func(offspring_a)
      offspring_b = mutation_func(offspring_b)
      next_generation += [offspring_a, offspring_b]

    population = next_generation

  population = sorted(
      population,
      key=lambda genome: fitness_func(genome),
      reverse=True
    )
  return population, i
start = time.time()
population, generations = run_evolution(
  population_func=partial(
    generate_population, size=10, genome_length=len(things)
  ),
  fitness_func=partial(
    fitness, things=more_things, weight_limit=3000
  ),
  fitness_limit=740,
  generation_limit=100
)
end = time.time()
print(f"Time taken: {end - start}")

print(f"Number of generations: {generations}")
print(f"Best solution: {population[0]}")

ValueError: genome and things must be of the same length

In [ ]:
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple

Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]


def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)


def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]


def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i
start = time.time()
population, generations = run_evolution(
    populate_func=partial(generate_population, size=10, genome_length=len(things)),
    fitness_func=partial(fitness, things=things, weight_limit=3000),
    fitness_limit=740,
    generation_limit=100,
    printer=print_stats
)
end = time.time()
print(f"Time taken: {end - start}")

print(f"Number of generations: {generations}")


GENERATION 00
Population: [11100, 11010, 10011, 10000, 01110, 01101, 01010, 01000, 00001, 11111]
Avg. Fitness: 333.000000
Best: 11100 (710.000000)
Worst: 11111 (0.000000)

GENERATION 01
Population: [11011, 11100, 11010, 11010, 11000, 10011, 10010, 10000, 00100, 10111]
Avg. Fitness: 513.000000
Best: 11011 (720.000000)
Worst: 10111 (0.000000)

GENERATION 02
Population: [11011, 11011, 11100, 11010, 11010, 11010, 11001, 10011, 10011, 10000]
Avg. Fitness: 654.000000
Best: 11011 (720.000000)
Worst: 10000 (500.000000)

GENERATION 03
Population: [11011, 11011, 11011, 11011, 11100, 10011, 10011, 10010, 10010, 01011]
Avg. Fitness: 603.000000
Best: 11011 (720.000000)
Worst: 01011 (220.000000)

GENERATION 04
Population: [11011, 11011, 11000, 10011, 10011, 10011, 01101, 01010, 11111, 11110]
Avg. Fitness: 423.000000
Best: 11011 (720.000000)
Worst: 11110 (0.000000)

GENERATION 05
Population: [11011, 11011, 11011, 11010, 10011, 10001, 10000, 01111, 10111, 10111]
Avg. Fitness: 473.000000
Best: 11011 (7